In [1]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [22]:
import coco, cv2, glob, h5py, imutils, json, math, matplotlib, os, PIL, random, re, skimage, sys, time, torch
import matplotlib.lines as lines
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.utils.data

from azure.storage import CloudStorageAccount
from azure.storage.blob import BlockBlobService
from matplotlib.patches import Polygon
from PIL import Image, ImageOps
from pyimagesearch.centroidtracker import CentroidTracker
from skimage import draw
from skimage.measure import find_contours
from torch.autograd import Variable
from torchvision import datasets
from tqdm import tqdm

# Import Mask R-CNN
TOP_DIR = os.path.abspath("../")
sys.path.append(TOP_DIR)
import fish_pytorch_style, utils, visualize
import model as modellib

from config import Config
from model import log

In [3]:
%matplotlib inline

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
%run AzureStorageHelper.ipynb

In [14]:
# Root directory of the project
TOP_DIR = os.path.abspath("../")

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory of images to run detection on
DATA_DIR = os.path.join(ROOT_DIR, 'frames')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')

### Notebook Preferences

In [7]:
def get_ax(rows=1, cols=1, size=12):
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return fig, ax

### List Training Runs

In [8]:
list_files(model_weights_share)

custom20180725T0702
custom20180728T1330
custom20180802T1151


### List Model Weights

In [9]:
folder = 'custom20180728T1330'

list_files(model_weights_share, folder)

mask_rcnn_custom_0001.pth
mask_rcnn_custom_0002.pth
mask_rcnn_custom_0003.pth
mask_rcnn_custom_0004.pth
mask_rcnn_custom_0005.pth
mask_rcnn_custom_0006.pth
mask_rcnn_custom_0007.pth
mask_rcnn_custom_0008.pth
mask_rcnn_custom_0009.pth
mask_rcnn_custom_0010.pth
mask_rcnn_custom_0011.pth
mask_rcnn_custom_0012.pth
mask_rcnn_custom_0013.pth
mask_rcnn_custom_0014.pth
mask_rcnn_custom_0015.pth
mask_rcnn_custom_0016.pth
mask_rcnn_custom_0017.pth
mask_rcnn_custom_0018.pth
mask_rcnn_custom_0019.pth
mask_rcnn_custom_0020.pth
mask_rcnn_custom_0021.pth
mask_rcnn_custom_0022.pth
mask_rcnn_custom_0023.pth
mask_rcnn_custom_0024.pth
mask_rcnn_custom_0025.pth
mask_rcnn_custom_0026.pth
mask_rcnn_custom_0027.pth
mask_rcnn_custom_0028.pth
mask_rcnn_custom_0029.pth
mask_rcnn_custom_0030.pth
mask_rcnn_custom_0031.pth
mask_rcnn_custom_0032.pth
mask_rcnn_custom_0033.pth
mask_rcnn_custom_0034.pth
mask_rcnn_custom_0035.pth
mask_rcnn_custom_0036.pth
mask_rcnn_custom_0037.pth
mask_rcnn_custom_0038.pth
mask_rcnn_cu

### Select Model Weights

In [10]:
file_name = 'mask_rcnn_custom_0160.pth'

download_path = os.path.join(os.getcwd(), file_name)

download_file(model_weights_share, folder, file_name, download_path, overwrite=True)

MODEL_WEIGHTS = download_path

Copying to /home/wonderwoman/Machine-Learning-Containers/ModelScoring_GPU/mask_rcnn_custom_0160.pth


### Test Set

In [11]:
download_path = './frames/test'

download_files(model_training_share, 'score', download_path=download_path)

In [12]:
class_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [13]:
class ModelConfig(Config):
    # GPU_COUNT * IMAGES_PER_GPU
    BATCH_SIZE = 2
    
    COCO_MODEL_PATH = MODEL_WEIGHTS
    
    DETECTION_MIN_CONFIDENCE = 0.70
    
    GPU_COUNT = 1
    
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1920
    
    LEARNING_RATE = 0.001
    
    NAME = "custom"

    # Number of classes (including background)
    NUM_CLASSES = 10 + 1
    
    # Necessary for docker image to optimize memory usage best
    NUM_WORKERS = 0
    
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 5
    
    VALIDATION_STEPS = 5
    
# Configurations
config = ModelConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[480 480]
 [240 240]
 [120 120]
 [ 60  60]
 [ 30  30]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_MODEL_PATH                /home/wonderwoman/Machine-Learning-Containers/ModelScoring_GPU/mask_rcnn_custom_0160.pth
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGENET_MODEL_PATH            /home/wonderwoman/Machine-Learning-Containers/ModelScoring_GPU/resnet50_imagenet.pth
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1920
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1920 1920    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL           

### Initialize Model

In [15]:
model = modellib.MaskRCNN(config=config, model_dir=MODEL_DIR)

if config.GPU_COUNT > 0:
    print('Cuda!', config.GPU_COUNT)
    model = model.cuda()

pretrained_state = torch.load(MODEL_WEIGHTS)
model_state = model.state_dict()

pretrained_state = { k:v for k,v in pretrained_state.items() if k in model_state and v.size() == model_state[k].size() }
model_state.update(pretrained_state)
model.load_state_dict(model_state)

Cuda! 1


### Score using Test Video

In [16]:
class InferenceConfig(ModelConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

inference_config = InferenceConfig()

In [17]:
def threshold_scores(results, minimum_score):
    rois = []
    masks = []
    class_ids = []
    scores = []
    
    r = results[0]
    
    for i in range(len(r['rois'])):
        score = r['scores'][i]
        
        if score >= minimum_score:
            rois.append(r['rois'][i])           
            masks.append(r['masks'][:,:,i])
            class_ids.append(r['class_ids'][i])
            scores.append(r['scores'][i])

    return np.array(rois), np.stack(masks, axis=-1), np.array(class_ids), np.array(scores)

In [18]:
def apply_mask(image, mask, color, alpha=0.2):
    for c in range(3):
        image[:, :, c] = np.where(mask == 1, image[:, :, c] * (1 - alpha) + alpha * color[c] * 255, image[:, :, c])
    return image

In [19]:
def display_instances(image, boxes, masks, class_ids, class_names, scores=None, fig=None, ax=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    figsize: (optional) the size of the image.
    """
    N = boxes.shape[0]
    
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')

    masked_image = image.astype(np.uint32).copy()

    for i in range(N):
        color = (1, 0, 1)

        # Bounding Box
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        
        p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, alpha=0.7, linestyle="dashed", edgecolor=color, facecolor='none')

        ax.add_patch(p)

        # Label
        class_id = class_ids[i]
        score = scores[i] if scores is not None else None
        label = class_names[class_id - 1]
        x = random.randint(x1, (x1 + x2) // 2)
        caption = "{} {:.3f}".format(label, score) if score else label
        ax.text(x1, y1 + 8, caption, color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        
        for verts in contours:
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)

    ax.imshow(masked_image.astype(np.uint8))
    
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    buffer = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    buffer.shape = (w, h, 3)
    plt.close(fig)
    
    return masked_image.astype(np.uint8), buffer

### Counts

In [23]:
centroidTracker = CentroidTracker()

In [37]:
videos = [file for file in os.listdir(download_path) if file.endswith('.MP4')]

for video in videos:
    reader = cv2.VideoCapture(os.path.join(download_path, video))
    writer = None
    
    frame_count = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    for i in tqdm(range(frame_count)):
        retval, image = reader.read()

        if retval:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            buffer = image
            
            try:
                results = model.detect([image])

                r = results[0]

                rois, masks, class_ids, scores = threshold_scores(results, 0.8)

                fig, ax=get_ax()
                
                image, buffer = display_instances(image, rois, masks, class_ids, class_names, scores, fig, ax)
            except Exception as error:
                print(error)
                
            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                writer = cv2.VideoWriter('Scored-' + video, fourcc, reader.get(cv2.CAP_PROP_FPS),(buffer.shape[1], buffer.shape[0]), True)

            writer.write(buffer)

    reader.release()
    reader = None
    
    writer.release()
    writer = None

 43%|████▎     | 4937/11472 [4:01:52<5:20:09,  2.94s/it]

 43%|████▎     | 4941/11472 [4:01:54<5:19:44,  2.94s/it]

 43%|████▎     | 4947/11472 [4:01:54<5:19:04,  2.93s/it]

 43%|████▎     | 4953/11472 [4:01:54<5:18:23,  2.93s/it]

 43%|████▎     | 4959/11472 [4:01:54<5:17:43,  2.93s/it]

...

100%|█████████▉| 11458/11472 [4:08:28<00:18,  1.30s/it]

100%|█████████▉| 11463/11472 [4:08:29<00:11,  1.30s/it]

100%|█████████▉| 11465/11472 [4:08:29<00:09,  1.30s/it]

100%|█████████▉| 11470/11472 [4:08:29<00:02,  1.30s/it]

100%|██████████| 11472/11472 [4:08:29<00:00,  1.30s/it]
